# Combining genotype info

### Load data
We need to combine the genotype info with the AngioTool data.


Start by loading the Excel spreadsheet with select columns (we don't care about the date, time, etc.) into a `pandas` DataFrame. 

In [3]:
import numpy as np
import pandas as pd

#tweaked_results = './processed/tweaked/TweakedResults11-6-2017.xls'
tweaked_results = '../data/TweakedResults2017-12-8.xls'
data = pd.read_excel(tweaked_results,
                     usecols = "A, G, N, O, Q, S")
# These are the Excel columns that correspond to 
# the desired AngioTool output parameters

#pd.options.display.float_format = '{:,.2f}'.format

Confirm you loaded the correct thing by looking at the header. You can always look at a Dataframe's first five entries through `df.head()`

In [4]:
data.head()

,Image Name,Vessel Thickness,Vessels percentage area,Total Number of Junctions,Total Vessels Length,Total Number of End Points
0,35-3b-field 1.tif,9,65.034498,119,14.215480,16
1,35-3b-field 2.tif,9,61.498248,183,16.877167,17
2,35-3b-field 3.tif,9,55.247350,157,15.500902,29
3,35-5b-field 1.tif,10,55.218967,138,14.465886,37
4,35-5-field 2.tif,9,51.731472,134,13.181119,32


### Create unique IDs for each animal
We need to parse the Image Name column in order to get a unique ID for each animal. The unique ID will be used for pairing genotypes using the Genotyping Key document.

I'm using list comprehensions to get Litter/ID info from the Image Name. Both are added to our `data` DataFrame.

In [5]:
data['Litter'] =   [x[:2] for x in data['Image Name']]
data['AnimalID'] = [x[:4] for x in data['Image Name']]

### Load genotyping key
Once again, we'll use the `read_excel` tool found within the `pandas` library.

In [7]:
key = pd.read_excel('../data/genotypingkey.xlsx')
key

,Litter,ID,AnimalID,Genotype
0,35,3,35-3,+/+
1,35,5,35-5,DEE/DEE
2,35,6,35-6,DEE/DEE
3,35,7,35-7,DEE/+
4,36,2,36-2,DEE/+
5,36,3,36-3,+/+
6,36,4,36-4,DEE/DEE
7,36,5,36-5,+/+
8,36,6,36-6,DEE/+
9,37,3,37-3,DEE/+


### Obtain entire dataset
We'll merge the two dataframes by using the `merge` function from the `pandas` library. Notice how `AnimalID` is used to match.

We can also drop unneeded columns such as `Image Name`, `ID`, and the duplicated `Litter_y`.

In [8]:
merged_inner = pd.merge(left = data, right = key,
                        how = 'inner',
                        left_on =  'AnimalID',
                        right_on = 'AnimalID'
                       )
## Drop unneeded columns
merged_inner.drop('Litter_y',         axis = 1, inplace = True)
merged_inner.drop('ID',               axis = 1, inplace = True)
merged_inner.drop('Image Name',       axis = 1, inplace = True)
merged_inner.drop('Vessel Thickness', axis = 1, inplace = True)

merged_inner.rename(columns = {'Litter_x':'Litter'},
                    inplace = True)


#pd.options.display.float_format = '{:,.2f}'.format

## Show first five observations
merged_inner

,Vessels percentage area,Total Number of Junctions,Total Vessels Length,Total Number of End Points,Litter,AnimalID,Genotype
0,65.034498,119,14.215480,16,35,35-3,+/+
1,61.498248,183,16.877167,17,35,35-3,+/+
2,55.247350,157,15.500902,29,35,35-3,+/+
3,55.218967,138,14.465886,37,35,35-5,DEE/DEE
4,51.731472,134,13.181119,32,35,35-5,DEE/DEE
5,53.907283,125,13.055196,47,35,35-5,DEE/DEE
6,62.635569,148,15.047376,38,36,36-4,DEE/DEE
7,65.269094,169,16.007724,27,36,36-4,DEE/DEE
8,60.138528,113,13.177062,28,36,36-4,DEE/DEE
9,62.868619,87,11.536940,27,36,36-4,DEE/DEE


In [9]:
## Round floats to 3 decimal points

merged_inner['Vessels percentage area'] = (merged_inner['Vessels percentage area']
                                           .apply(lambda x: round(x, 3)))
merged_inner['Total Vessels Length'] = (merged_inner['Total Vessels Length']
                                        .apply(lambda x: round(x, 3)))
merged_inner

,Vessels percentage area,Total Number of Junctions,Total Vessels Length,Total Number of End Points,Litter,AnimalID,Genotype
0,65.034,119,14.215,16,35,35-3,+/+
1,61.498,183,16.877,17,35,35-3,+/+
2,55.247,157,15.501,29,35,35-3,+/+
3,55.219,138,14.466,37,35,35-5,DEE/DEE
4,51.731,134,13.181,32,35,35-5,DEE/DEE
5,53.907,125,13.055,47,35,35-5,DEE/DEE
6,62.636,148,15.047,38,36,36-4,DEE/DEE
7,65.269,169,16.008,27,36,36-4,DEE/DEE
8,60.139,113,13.177,28,36,36-4,DEE/DEE
9,62.869,87,11.537,27,36,36-4,DEE/DEE


In [10]:
## Save to a csv file
## 
merged_inner.to_csv("../data/unblinded_data.csv", index = False)